In [127]:
import sys
sys.executable

'/Users/b/bio/cardiac/model1/new_env/bin/python'

In [128]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import pandas as pd
import spacy
import datasets
from datasets import Dataset
import tqdm
import evaluate
from collections import Counter, defaultdict
import pyabf
import ast
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker